# Streaming and Batching with Chat Models

**Purpose:**  
Illustrate how to use streaming and batching with a chat model backend, and how concurrency controls affect batch execution behavior.


## Core concepts

**Streaming**  
Streaming returns partial model outputs incrementally as tokens or chunks are generated. This reduces perceived latency and enables real-time processing or UI updates before the full response is complete.

**Batching**  
Batching sends multiple prompts in a single logical operation. The runtime schedules and executes them, potentially in parallel, improving throughput and reducing overhead when issuing many independent requests.

**Maximum concurrency**  
`max_concurrency` limits how many batch requests may be in-flight at the same time. This is a backpressure mechanism used to control resource usage, avoid provider rate limits, and stabilize latency under load.


In [ ]:
# Environment setup

import os
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [ ]:
# Model initialization

from langchain_groq import ChatGroq

llm= ChatGroq(model="qwen/qwen3-32b")

In [ ]:
# Synchronous invocation (baseline)

llm.invoke("Why do parrots have colorful feathers?")

AIMessage(content='<think>\nOkay, so I need to figure out why parrots have such colorful feathers. Let me start by recalling what I know. Parrots are known for their bright colors—greens, yellows, blues, reds, etc. Maybe it\'s about attracting mates? I remember in some animals, like peacocks, males show off their feathers to attract females. So maybe parrots do something similar. But there\'s probably more to it than that.\n\nAnother thought: maybe the colors help them blend into their environment. Wait, but if they\'re so colorful, wouldn\'t that make them stand out? Parrots often live in places with lots of green foliage, like rainforests. So maybe their green feathers help them blend in. But then why do they have so many other colors too? Maybe different colors serve different purposes.\n\nAlso, could it be about communication? Birds might use colors to signal to others of their species. Like, showing they\'re healthy or strong. There\'s the idea of sexual selection where brighter c

In [ ]:
# Streaming example
# The model yields partial output chunks as they are produced.

for chunk in llm.stream("Why do parrots have colorful feathers?"):
    print(chunk.text, end="|", flush=True)

|<think>|
|Okay|,| so| I| need| to| figure| out| why| par|rots| have| such| colorful| feathers|.| Let| me| start| by| recalling| what| I| know|.| I| remember| that| animals| often| have| bright| colors| for| specific| reasons|,| like| attracting| mates| or| warning| predators|.| But| par|rots| are| birds|,| and| their| feathers| are| really| vibrant|.| Maybe| there|'s| a| mix| of| reasons| here|.

|First| thought|:| sexual| selection|.| In| many| bird| species|,| males| have| brighter| colors| to| attract| females|.| For| example|,| pe|ac|ocks| have| those| elaborate| tails|.| So| maybe| male| par|rots| have| brighter| feathers| to| show| off| to| females|.| But| not| all| par|rots| are| male|,| right|?| Some| females| have| colorful| feathers| too|.| Maybe| it|'s| mutual| ornament|ation|.| I|'ve| heard| that| in| some| species|,| both| males| and| females| are| colorful|,| which| could| mean| that| both| sexes| choose| mates| based| on| color|.| So| maybe| it|'s| not| just| about| mal

In [ ]:
# Basic batching
# Prompts are executed as a group and responses are returned in order.

responses = llm.batch([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
])
for response in responses:
    print(response)

content="<think>\nOkay, so I need to figure out why parrots have colorful feathers. Let me start by recalling what I know about parrots. They're known for their bright colors—greens, blues, reds, yellows, etc. I remember that in animals, bright colors can serve different purposes like attracting mates, camouflage, or warning predators. But parrots are usually in environments like rainforests, which are green and leafy. So maybe their colors help them blend in? Wait, but some parrots are very bright, which might make them stand out. That's confusing. Maybe the colors have more to do with communication or mating. \n\nI think about other birds, like peacocks, which have elaborate tails for attracting mates. Maybe parrots use their colors similarly. Also, some birds have colors that indicate their health or fitness. Like, if a bird can maintain bright feathers despite parasites or disease, it might be a sign of good genes. That's part of sexual selection. \n\nAnother angle is that parrots 

In [ ]:
# Batched execution with bounded concurrency
# Only 'max_concurrency' requests are allowed to execute in parallel at any time.

llm.batch(
    ["Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?",
    "Why do parrots have red beaks?",
    "What is the meaning of life?",
    "why is the sky blue?",
    "why black holes are black?"],
    config={
        'max_concurrency': 5,  # Limit to 5 parallel calls
    }
)

[AIMessage(content="<think>\nOkay, so I need to figure out why parrots have colorful feathers. Let's start by thinking about what I know about parrots. They're birds, right? And they're known for their bright colors—greens, blues, yellows, reds, etc. But why? I remember from biology that animals can have colorful features for various reasons like attracting mates, camouflage, or communication.\n\nFirst, maybe it's about attracting mates. Like, in many bird species, males have brighter colors to show they're healthy and good genes. But wait, parrots are often colorful in both males and females. So maybe it's not just for attracting the opposite sex. Or could it be that both genders use color in some way?\n\nAnother thought: maybe the colors help in communication. Parrots are social animals, so maybe their colors help them recognize each other or signal their mood. For example, some birds can change their color slightly when they're agitated. Do parrots do that?\n\nThen there's the idea 

## Summary

- Streaming emits partial outputs early and continuously, reducing perceived latency and enabling real-time handling.
- Batching improves throughput by grouping independent prompts into a single scheduled operation.
- `max_concurrency` provides backpressure, preventing overload and helping comply with provider rate limits or infrastructure constraints.
